In [12]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
from matplotlib import gridspec
from scipy import stats, interpolate
import os
from tensorflow.keras import backend as K  

import tqdm



In [15]:
alpha = 0.1

def load_dataT21(input_frame):
    dataset = input_frame[["mj1","tau1j1","tau2j1","mj2","tau1j2","tau2j2"]]
    dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
    dataset["mjTwo"] = dataset[["mj1", "mj2"]].max(axis=1)
    dataset["mjOne"] = dataset[["mj1", "mj2"]].min(axis=1)

    dataset["tau1jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau1jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau21jOne"] = dataset["tau2jOne"]/dataset["tau1jOne"]
    dataset["tau21jTwo"] = dataset["tau2jTwo"]/dataset["tau1jTwo"]
    dataset["mjTwo"] = dataset["mjTwo"]/1000. + 2*alpha*dataset["mjj"]
    dataset["mjOne"] = dataset["mjOne"]/1000. + alpha*dataset["mjj"]
    dataset["mjDelta"] = (dataset["mjTwo"] - dataset["mjOne"])

    dataset = dataset.fillna(0)
    dataset = dataset[["mjj","mjOne","mjDelta", "tau21jOne", "tau21jTwo"]]
    return dataset.to_numpy()

def load_dataT32(input_frame):
    dataset = input_frame[["mj1","tau1j1","tau2j1","mj2","tau1j2","tau2j2","tau3j1","tau3j2"]]
    dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
    dataset["mjTwo"] = dataset[["mj1", "mj2"]].max(axis=1)
    dataset["mjOne"] = dataset[["mj1", "mj2"]].min(axis=1)

    dataset["tau2jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau3jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau3jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau32jOne"] = dataset["tau3jOne"]/dataset["tau2jOne"]
    dataset["tau32jTwo"] = dataset["tau3jTwo"]/dataset["tau2jTwo"] 
    dataset["mjTwo"] = dataset["mjTwo"]/1000. + 2*alpha*dataset["mjj"]
    dataset["mjOne"] = dataset["mjOne"]/1000. + alpha*dataset["mjj"]
    dataset["mjDelta"] = (dataset["mjTwo"] - dataset["mjOne"])

    dataset = dataset.fillna(0)
    dataset = dataset[["mjj","mjOne","mjDelta", "tau32jOne", "tau32jTwo"]]
    return dataset.to_numpy()

def load_data_more(input_frame):
    dataset = input_frame[["mj1","tau1j1","tau2j1","mj2","tau1j2","tau2j2","tau3j1","tau3j2"]]
    
    #Masa ya al cuadrado
    dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
    
    
    dataset["mjTwo"] = dataset[["mj1", "mj2"]].max(axis=1)
    dataset["mjOne"] = dataset[["mj1", "mj2"]].min(axis=1)

    dataset["tau1jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau1jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau21jOne"] = dataset["tau2jOne"]/dataset["tau1jOne"]
    dataset["tau21jTwo"] = dataset["tau2jTwo"]/dataset["tau1jTwo"]
    
    #tau32
    dataset["tau3jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau3jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau32jOne"] = dataset["tau3jOne"]/dataset["tau2jOne"]
    dataset["tau32jTwo"] = dataset["tau3jTwo"]/dataset["tau2jTwo"]   
    
    #pT
    dataset["pT1"] = (input_frame["pxj1"]**2+input_frame["pyj1"]**2)**0.5/1000.
    dataset["pT2"] = (input_frame["pxj2"]**2+input_frame["pyj2"]**2)**0.5/1000.
    dataset["pTjOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["pT1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["pT2"]
    dataset["pTjTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["pT1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["pT2"]    
    
    #eta
    dataset["E1"] = ((1000*dataset["pT1"])**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5/1000.
    dataset["E2"] = ((1000*dataset["pT2"])**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5/1000.
    dataset["y1"] = 0.5*np.log((dataset["E1"] + input_frame["pzj1"]/1000.)/(dataset["E1"] - input_frame["pzj1"]/1000.))
    dataset["y2"] = 0.5*np.log((dataset["E2"] + input_frame["pzj2"]/1000.)/(dataset["E2"] - input_frame["pzj2"]/1000.))
    dataset["Dy"] = dataset["y1"]-dataset["y2"]
    
    dataset["mjTwo"] = dataset["mjTwo"]/1000. + 2*alpha*dataset["mjj"]
    dataset["mjOne"] = dataset["mjOne"]/1000. + alpha*dataset["mjj"]
    dataset["mjDelta"] = (dataset["mjTwo"] - dataset["mjOne"])

    dataset = dataset.fillna(0)
    dataset = dataset[["mjj","mjOne","mjDelta", "tau21jOne", "tau21jTwo", "tau32jOne", "tau32jTwo","pTjOne","pTjTwo","Dy"]]
    return dataset.to_numpy()

column_labelsT21 = [
    r"$m_{JJ}$",
    r"$m^{J_1}$",
    "$m^{J_2} - m^{J_1}$",
    r"$\tau_{2,1}^{J_1}$", 
    r"$\tau_{2,1}^{J_2}$",
]

column_labelsT32 = [
    r"$m_{JJ}$",
    r"$m^{J_1}$",
    "$m^{J_2} - m^{J_1}$",
    r"$\tau_{3,2}^{J_1}$", 
    r"$\tau_{3,2}^{J_2}$",
]

In [16]:
#original_data=pd.read_hdf("C:/Users/knukl/Desktop/FINAL/input/events_anomalydetection_v2.features.h5")
original_data=pd.read_hdf("/Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoLa_ANN_Analysis/content/events_anomalydetection_v2.features.h5")
features_sig=original_data.query("label == 1")
features_bg=original_data.query("label == 0")


features_bg2=pd.read_hdf("/Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoLa_ANN_Analysis/content/events_anomalydetection_DelphesHerwig_qcd_features.h5")

In [17]:
dataset_bg=load_dataT21(features_bg)
dataset_bg2=load_dataT21(features_bg2)
dataset_sig=load_dataT21(features_sig)

dataset_bgT32=load_dataT32(features_bg)
dataset_bg2T32=load_dataT32(features_bg2)
dataset_sigT32=load_dataT32(features_sig)

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\259476259.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\259476259.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\259476259.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["tau21jOne"] = dataset["tau2jOne"]/dataset["tau1jOne"]
C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\259476259.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(inp

In [18]:
dataset_bg_more=load_data_more(features_bg)
dataset_bg2_more=load_data_more(features_bg2)
dataset_sig_more=load_data_more(features_sig)

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\259476259.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\259476259.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [19]:
dataset_bg

array([[3.30721939, 0.36961794, 0.52971994, 0.58331666, 0.26323715],
       [3.1076206 , 0.33376126, 0.67729487, 0.78773201, 0.51908614],
       [3.00489527, 0.37264503, 0.30656472, 0.78958448, 0.91180871],
       ...,
       [4.283545  , 0.4848261 , 0.7679949 , 0.71013539, 0.83904146],
       [2.78405802, 0.3835928 , 0.53700979, 0.6447025 , 0.22796556],
       [2.92645683, 0.34226608, 0.39324028, 0.63616232, 0.91674757]])

In [20]:
from sklearn.model_selection import train_test_split

SR_low = 3.3
SR_high = 3.7
test_size = 0.5
EPOCHS=20
batch_size = 200
SB_width = 0.3

def sr_crit(d):
    return (d[:,0] < SR_high) & (d[:,0] >= SR_low)



bg_srsb, bg2_srsb, sig_srsb = sr_crit(dataset_bg_more), sr_crit(dataset_bg2_more), sr_crit(dataset_sig_more)

SR_background_data = dataset_bg[bg_srsb]
SB_background_data = dataset_bg[~bg_srsb]
print(~bg_srsb)

SR_background_sim = dataset_bg2[bg2_srsb]
SB_background_sim = dataset_bg2[~bg2_srsb]

SR_signal = dataset_sig[sig_srsb]
SB_signal = dataset_sig[~sig_srsb]

N_inputs = len(SR_background_data.T) - 1


((SR_background_data, SR_background_data_fortest),
 (SR_background_sim, SR_background_sim_fortest),
 (SB_background_data, SB_background_data_fortest),
 (SB_background_sim, SB_background_sim_fortest),
 ) = [train_test_split(arr, test_size=test_size) for arr in [
    SR_background_data, SR_background_sim,
    SB_background_data, SB_background_sim,
]]

mn,mx = np.percentile(np.concatenate([SB_background_data, SB_background_sim, SR_background_data, 
                                      SR_background_sim]), [1,99], axis=0)

def norm_func(d):
    return (d - mn)/(mx - mn)

Nsig = 1000
Nsig_SB = int(np.round(len(SB_signal)*Nsig/len(SR_signal)))

fmt = '{:>20}:  {}'
print(fmt.format('# signal events', Nsig))
print(fmt.format('signal significance', len(SR_signal[0:Nsig])/len(SR_background_data)**0.5))
print(fmt.format('s/b ratio', len(SR_signal[0:Nsig])/len(SR_background_data)))

n_injections = 20
SR_signals_to_inject = [None] * n_injections
SB_signals_to_inject = [None] * n_injections
signals_to_test = [None] * n_injections

for i in range(n_injections):
    idx = np.isin(range(len(SR_signal)), np.random.choice(SR_signal.shape[0], size=Nsig, replace=False))
    SR_signals_to_inject[i] = SR_signal[idx, :]
    signals_to_test[i] = SR_signal[~idx, :]
    
    idx = np.isin(range(len(SB_signal)), np.random.choice(SB_signal.shape[0], size=Nsig_SB, replace=False))
    SB_signals_to_inject[i] = SB_signal[idx, :]

[False  True  True ...  True  True  True]
     # signal events:  1000
 signal significance:  4.059677443835412
           s/b ratio:  0.016480980947986026


In [21]:
#Next is CWoLa
tpr_pts = np.linspace(0, 1, 10000)
cwola_rocs = pd.DataFrame(columns=['fpr', 'tpr'])

for i in tqdm.tqdm(range(n_injections)):
    
    SR_signal_to_inject = SR_signals_to_inject[i]
    SB_signal_to_inject = SB_signals_to_inject[i]

    signal_to_test = signals_to_test[i]
        
    X_cwola_train = np.concatenate([SR_signal_to_inject, SR_background_data, SB_signal_to_inject, SB_background_data])
    Y_cwola_train = np.concatenate([np.ones(len(SR_signal_to_inject)), np.ones(len(SR_background_data)),
                                    np.zeros(len(SB_signal_to_inject)), np.zeros(len(SB_background_data))])
    
    X_cwola_train, Y_cwola_train = shuffle(X_cwola_train, Y_cwola_train, )

    sbmjj = X_cwola_train[:,0]
    sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))
    
    X_cwola_train = X_cwola_train[sb_crit]
    Y_cwola_train = Y_cwola_train[sb_crit]
    
    w_low = 0.5*(len(SR_signal_to_inject) + len(SR_background_data))/((X_cwola_train[:,0] <= SR_low).sum())
    w_high = 0.5*(len(SR_signal_to_inject) + len(SR_background_data))/((X_cwola_train[:,0] >= SR_high).sum())

    W_cwola_train = np.ones_like(Y_cwola_train)
    W_cwola_train[(Y_cwola_train == 0) & (X_cwola_train[:,0] <= SR_low)] = w_low
    W_cwola_train[(Y_cwola_train == 0) & (X_cwola_train[:,0] >= SR_high)] = w_high
    
    X_cwola_val = np.concatenate([signal_to_test,SR_background_data_fortest])
    Y_cwola_val = np.concatenate([np.ones(len(signal_to_test)),np.zeros(len(SR_background_data_fortest))])
    
    K.clear_session()
    model_cwola = Sequential()
    model_cwola.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    model_cwola.add(Dense(64, activation='relu'))
    model_cwola.add(Dense(64, activation='relu'))
    model_cwola.add(Dense(1, activation='sigmoid'))
    model_cwola.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwola = model_cwola.fit(X_cwola_train[:,1:], Y_cwola_train, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_train)

    scores_cwola = model_cwola.predict(X_cwola_val[:,1:],batch_size=batch_size)
        
    fpr_cwola, tpr_cwola, _ = roc_curve(Y_cwola_val, scores_cwola)
    cwola_rocs = cwola_rocs.append({'fpr': fpr_cwola, 'tpr': tpr_cwola}, ignore_index=True)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2394776141.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cwola_rocs = cwola_rocs.append({'fpr': fpr_cwola, 'tpr': tpr_cwola}, ignore_index=True)
  5%|████▏                                                                              | 1/20 [01:24<26:51, 84.82s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2394776141.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cwola_rocs = cwola_rocs.append({'fpr': fpr_cwola, 'tpr': tpr_cwola}, ignore_index=True)
 10%|████████▎                                                                          | 2/20 [02:52<25:53, 86.29s/it]

...

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2394776141.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cwola_rocs = cwola_rocs.append({'fpr': fpr_cwola, 'tpr': tpr_cwola}, ignore_index=True)
 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [27:18<03:05, 92.82s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2394776141.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cwola_rocs = cwola_rocs.append({'fpr': fpr_cwola, 'tpr': tpr_cwola}, ignore_index=True)
 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [29:00<01:35, 95.50s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2394776141.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cwola_rocs = cwola_rocs.append({'fpr': fpr_cwola, 'tpr': tpr_cwola}, ignore_index=True)
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [30:38<00:00, 91.95s/it]


In [22]:
# CWoLa
fpr_interp = [None] * len(cwola_rocs)

for i, row in cwola_rocs.iterrows():
    fpr, tpr = row
    interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

    fpr_pts = interp(tpr_pts)
    fpr_interp[i] = fpr_pts

fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

cwola_max = np.nanmax(fpr_interp, axis=0).data
cwola_min = np.nanmin(fpr_interp, axis=0).data
cwola_med = np.median(fpr_interp, axis=0).data
cwola_mean = np.nanmean(fpr_interp, axis=0).data
cwola_std = np.nanstd(fpr_interp, axis=0).data
    
sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
cwola_smean = np.nanmean(sig_interp, axis=0).data
cwola_sstd = np.nanstd(sig_interp, axis=0).data

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\3456826973.py:11: RuntimeWarning: divide by zero encountered in true_divide
  fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))
C:\Users\bolit\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [23]:
#Now let's optimal CWoLa
ocwola_rocs = pd.DataFrame(columns=['fpr', 'tpr'])

for i in tqdm.tqdm(range(n_injections)):
    SR_signal_to_inject = SR_signals_to_inject[i]
    SB_signal_to_inject = SB_signals_to_inject[i]

    signal_to_test = signals_to_test[i]

    X_ocwola_train = np.concatenate([SR_signal_to_inject, SR_background_data,SR_background_data_fortest])
    Y_ocwola_train = np.concatenate([np.ones(len(SR_signal_to_inject)),np.ones(len(SR_background_data)),np.zeros(len(SR_background_data_fortest))])

    X_ocwola_train, Y_ocwola_train = shuffle(X_ocwola_train, Y_ocwola_train)

    sbmjj = X_ocwola_train[:,0]
    sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))
    
    X_ocwola_train = X_ocwola_train[sb_crit]
    Y_ocwola_train = Y_ocwola_train[sb_crit]

    X_ocwola_val = np.concatenate([signal_to_test,SR_background_data_fortest])
    Y_ocwola_val = np.concatenate([np.ones(len(signal_to_test)),np.zeros(len(SR_background_data_fortest))])


    K.clear_session()
    model_ocwola = Sequential()
    model_ocwola.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    
    model_ocwola.add(Dense(64, activation='relu'))
    model_ocwola.add(Dense(64, activation='relu'))
    model_ocwola.add(Dense(1, activation='sigmoid'))
    model_ocwola.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    hist_ocwola = model_ocwola.fit(X_ocwola_train[:,1:], Y_ocwola_train, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0)

    scores_ocwola = model_ocwola.predict(X_ocwola_val[:,1:],batch_size=batch_size)
    
    fpr_ocwola, tpr_ocwola, _ = roc_curve(Y_ocwola_val, scores_ocwola)
    ocwola_rocs = ocwola_rocs.append({'fpr': fpr_ocwola, 'tpr': tpr_ocwola}, ignore_index=True)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\887101774.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ocwola_rocs = ocwola_rocs.append({'fpr': fpr_ocwola, 'tpr': tpr_ocwola}, ignore_index=True)
  5%|████▏                                                                              | 1/20 [00:25<07:59, 25.25s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\887101774.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ocwola_rocs = ocwola_rocs.append({'fpr': fpr_ocwola, 'tpr': tpr_ocwola}, ignore_index=True)
 10%|████████▎                                                                          | 2/20 [00:48<07:09, 23.85s/it]

...

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\887101774.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ocwola_rocs = ocwola_rocs.append({'fpr': fpr_ocwola, 'tpr': tpr_ocwola}, ignore_index=True)
 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [07:25<00:51, 25.51s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\887101774.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ocwola_rocs = ocwola_rocs.append({'fpr': fpr_ocwola, 'tpr': tpr_ocwola}, ignore_index=True)
 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [07:50<00:25, 25.53s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\887101774.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ocwola_rocs = ocwola_rocs.append({'fpr': fpr_ocwola, 'tpr': tpr_ocwola}, ignore_index=True)
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:16<00:00, 24.82s/it]


In [24]:
# optimal CWoLa
tpr_pts = np.linspace(0, 1, 10000)

fpr_interp = [None] * len(ocwola_rocs)
    
for i, row in ocwola_rocs.iterrows():
    fpr, tpr = row
    interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

    fpr_pts = interp(tpr_pts)
    fpr_interp[i] = fpr_pts

fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

ocwola_max = np.nanmax(fpr_interp, axis=0).data
ocwola_min = np.nanmin(fpr_interp, axis=0).data
ocwola_med = np.median(fpr_interp, axis=0).data
ocwola_mean = np.nanmean(fpr_interp, axis=0).data
ocwola_std = np.nanstd(fpr_interp, axis=0).data
   
sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
ocwola_smean = np.nanmean(sig_interp, axis=0).data
ocwola_sstd = np.nanstd(sig_interp, axis=0).data

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2785028550.py:13: RuntimeWarning: divide by zero encountered in true_divide
  fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))
C:\Users\bolit\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [25]:
#Next is SA-CWoLa
myklambda = 1 #np.round(len(SR_background_data)/len(SR_background_sim), 3)

ksacwola_rocs = pd.DataFrame(columns=['fpr', 'tpr'])

for i in tqdm.tqdm(range(n_injections)):
    
    SR_signal_to_inject = SR_signals_to_inject[i]
    SB_signal_to_inject = SB_signals_to_inject[i]

    signal_to_test = signals_to_test[i]

    X_ksacwola_train = np.concatenate([
        SR_signal_to_inject, SB_signal_to_inject[SB_signal_to_inject[:,0] < SR_low], SB_signal_to_inject[SB_signal_to_inject[:,0] > SR_high],
        SR_background_data, SB_background_data[SB_background_data[:,0] < SR_low], SB_background_data[SB_background_data[:,0] > SR_high],
        SR_background_sim, SB_background_sim[SB_background_sim[:,0] < SR_low], SB_background_sim[SB_background_sim[:,0] > SR_high]
    ])
    Y_ksacwola_train = np.concatenate([
        np.ones(len(SR_signal_to_inject)), np.zeros(len(SB_signal_to_inject)),
        np.ones(len(SR_background_data)), np.zeros(len(SB_background_data)),
        np.zeros(len(SR_background_sim)), np.ones(len(SB_background_sim))
    ])
        
    n_low_data = ((SB_background_data[:,0] > (SR_low - SB_width)) & ((SB_background_data[:,0] <= SR_low))).sum()
    n_low_sim = ((SB_background_sim[:,0] > (SR_low - SB_width)) & ((SB_background_sim[:,0] <= SR_low))).sum()
    n_low_sig = ((SB_signal_to_inject[:,0] > (SR_low - SB_width)) & ((SB_signal_to_inject[:,0] <= SR_low))).sum()

    n_hi_data = ((SB_background_data[:,0] < (SR_high + SB_width)) & ((SB_background_data[:,0] >= SR_high))).sum()
    n_hi_sim = ((SB_background_sim[:,0] < (SR_high + SB_width)) & ((SB_background_sim[:,0] >= SR_high))).sum()
    n_hi_sig = ((SB_signal_to_inject[:,0] < (SR_high + SB_width)) & ((SB_signal_to_inject[:,0] >= SR_high))).sum()

    n_sr_data = len(SR_background_data)
    n_sr_sim = len(SR_background_sim)
    n_sr_sig = len(SR_signal_to_inject)
        
    
    w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
    w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)
    
    w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
    w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
    
    
    
    W_ksacwola_train = np.concatenate([
        np.ones(len(SR_signal_to_inject)), w_low*np.ones((SB_signal_to_inject[:,0] <= SR_low).sum()), w_high*np.ones((SB_signal_to_inject[:,0] > SR_high).sum()),
        np.ones(len(SR_background_data)), w_low*np.ones((SB_background_data[:,0] <= SR_low).sum()), w_high*np.ones((SB_background_data[:,0] > SR_high).sum()),
        myklambda*np.ones(len(SR_background_sim)), w_low_sim*myklambda*np.ones((SB_background_sim[:,0] <= SR_low).sum()), w_high_sim*myklambda*np.ones((SB_background_sim[:,0] > SR_high).sum())
    ])
    
    sbmjj = X_ksacwola_train[:,0]
    sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))
    
    X_ksacwola_train = X_ksacwola_train[sb_crit]
    Y_ksacwola_train = Y_ksacwola_train[sb_crit]
    W_ksacwola_train = W_ksacwola_train[sb_crit]
    
    
    X_ksacwola_train, Y_ksacwola_train, W_ksacwola_train = shuffle(X_ksacwola_train, Y_ksacwola_train, W_ksacwola_train)

    
    X_ksacwola_val = np.concatenate([signal_to_test,SR_background_data_fortest])
    Y_ksacwola_val = np.concatenate([np.ones(len(signal_to_test)),np.zeros(len(SR_background_data_fortest))])
    
    K.clear_session()
    model_ksacwola = Sequential()
    model_ksacwola.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    model_ksacwola.add(Dense(64, activation='relu'))
    model_ksacwola.add(Dense(64, activation='relu'))
    model_ksacwola.add(Dense(1, activation='sigmoid'))
    model_ksacwola.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_ksacwola = model_ksacwola.fit(X_ksacwola_train[:,1:], Y_ksacwola_train, epochs=EPOCHS, 
                                     batch_size=batch_size,sample_weight=W_ksacwola_train,
                                     verbose=0
                                    )

    scores_ksacwola = model_ksacwola.predict(X_ksacwola_val[:,1:],batch_size=batch_size)
    
    fpr_ksacwola, tpr_ksacwola, _ = roc_curve(Y_ksacwola_val, scores_ksacwola)
    ksacwola_rocs = ksacwola_rocs.append({'fpr': fpr_ksacwola, 'tpr': tpr_ksacwola}, ignore_index=True)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\210455975.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksacwola_rocs = ksacwola_rocs.append({'fpr': fpr_ksacwola, 'tpr': tpr_ksacwola}, ignore_index=True)
  5%|████                                                                              | 1/20 [03:06<58:59, 186.31s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\210455975.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksacwola_rocs = ksacwola_rocs.append({'fpr': fpr_ksacwola, 'tpr': tpr_ksacwola}, ignore_index=True)
 10%|████████▏                                                                         | 2/20 [06:03<54:18, 181.05s/it]

...

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\210455975.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksacwola_rocs = ksacwola_rocs.append({'fpr': fpr_ksacwola, 'tpr': tpr_ksacwola}, ignore_index=True)
 90%|████████████████████████████████████████████████████████████████████████▉        | 18/20 [54:33<06:10, 185.02s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\210455975.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksacwola_rocs = ksacwola_rocs.append({'fpr': fpr_ksacwola, 'tpr': tpr_ksacwola}, ignore_index=True)
 95%|████████████████████████████████████████████████████████████████████████████▉    | 19/20 [57:26<03:01, 181.38s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\210455975.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ksacwola_rocs = ksacwola_rocs.append({'fpr': fpr_ksacwola, 'tpr': tpr_ksacwola}, ignore_index=True)
100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:00:20<00:00, 181.03s/it]


In [26]:
#Next is SALAD
SALAD_rocs = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injections)):
    SR_signal_to_inject = SR_signals_to_inject[i]
    SB_signal_to_inject = SB_signals_to_inject[i]
    signal_to_test = signals_to_test[i]
    
    n_data_sb = len(SB_background_sim) - len(SB_signal_to_inject)
    n_data_sr = len(SR_background_sim) - len(SR_signal_to_inject)

    X_SALAD_sb_train = np.concatenate([SB_background_sim, SB_background_data[np.random.choice(len(SB_background_data), n_data_sb, replace=False)], SB_signal_to_inject])
    Y_SALAD_sb_train = np.concatenate([np.zeros(len(SB_background_sim)), np.ones(n_data_sb), np.ones(len(SB_signal_to_inject))])
    
    sbmjj = X_SALAD_sb_train[:,0]
    sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

    X_SALAD_sb_train = X_SALAD_sb_train[sb_crit]
    Y_SALAD_sb_train = Y_SALAD_sb_train[sb_crit]
    
    N_sb_low = (X_SALAD_sb_train[Y_SALAD_sb_train == 1][:,0] <= SR_low).sum()    
    N_sb_high = (X_SALAD_sb_train[Y_SALAD_sb_train == 1][:,0] >= SR_high).sum()    
    w_low = 0.5*(len(SR_signal_to_inject) + n_data_sr)/((SB_signal_to_inject[:,0] <= SR_low).sum() + N_sb_low)
    w_high = 0.5*(len(SR_signal_to_inject) + n_data_sr)/((SB_signal_to_inject[:,0] >= SR_high).sum() + N_sb_high)
    
    W_SALAD_sb_train = np.ones_like(Y_SALAD_sb_train)
    W_SALAD_sb_train[X_SALAD_sb_train[:,0]<=SR_low] *= w_low
    W_SALAD_sb_train[X_SALAD_sb_train[:,0]>=SR_high] *= w_high
    
    
    X_SALAD_sr_train = np.concatenate([SR_signal_to_inject, SR_background_sim, SR_background_data[np.random.choice(len(SR_background_data), n_data_sr, replace=False)]])
    Y_SALAD_sr_train = np.concatenate([np.ones(len(SR_signal_to_inject)), np.zeros(len(SR_background_sim)), np.ones(n_data_sr)])
    
    X_SALAD_sb_train, Y_SALAD_sb_train, W_SALAD_sb_train = shuffle(X_SALAD_sb_train, Y_SALAD_sb_train, W_SALAD_sb_train)
    X_SALAD_sr_train, Y_SALAD_sr_train = shuffle(X_SALAD_sr_train, Y_SALAD_sr_train)
    
    X_SALAD_val = np.concatenate([signal_to_test, SR_background_data_fortest])
    Y_SALAD_val = np.concatenate([np.ones(len(signal_to_test)),np.zeros(len(SR_background_data_fortest))])
    
    K.clear_session()
    model_SALAD_sb = Sequential()
    model_SALAD_sb.add(Dense(64, input_dim=N_inputs + 1, activation='relu')) 
    model_SALAD_sb.add(Dense(64, activation='relu'))
    model_SALAD_sb.add(Dense(64, activation='relu'))
    model_SALAD_sb.add(Dense(1, activation='sigmoid'))
    model_SALAD_sb.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'],)

    hist_SALAD_sb = model_SALAD_sb.fit(
        X_SALAD_sb_train, Y_SALAD_sb_train, epochs=EPOCHS,
        batch_size=batch_size,
        verbose=0, sample_weight=W_SALAD_sb_train
    )

    def get_weights(data, model):
        yhat = model.predict(data, batch_size=int(0.1*len(data)))
        return np.squeeze(yhat/(1 - yhat))

    # apply weight function
    W_SALAD_sr_train = np.ones_like(Y_SALAD_sr_train)
    W_SALAD_sr_train[Y_SALAD_sr_train == 0] = get_weights(X_SALAD_sr_train[Y_SALAD_sr_train == 0], model_SALAD_sb)

    # run discriminating model
    model_SALAD_sr = Sequential()
    model_SALAD_sr.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    model_SALAD_sr.add(Dense(64, activation='relu'))
    model_SALAD_sr.add(Dense(64, activation='relu'))
    model_SALAD_sr.add(Dense(1, activation='sigmoid'))
    model_SALAD_sr.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_SALAD_sr = model_SALAD_sr.fit(
        X_SALAD_sr_train[:, 1:], Y_SALAD_sr_train, epochs=EPOCHS,
        batch_size=batch_size, verbose=0,
        sample_weight=W_SALAD_sr_train
    )

    scores_SALAD = model_SALAD_sr.predict(X_SALAD_val[:, 1:], batch_size=batch_size)

    fpr_SALAD, tpr_SALAD, _ = roc_curve(Y_SALAD_val, scores_SALAD)
    SALAD_rocs = SALAD_rocs.append({'fpr': fpr_SALAD, 'tpr': tpr_SALAD}, ignore_index=True)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\345227218.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SALAD_rocs = SALAD_rocs.append({'fpr': fpr_SALAD, 'tpr': tpr_SALAD}, ignore_index=True)
  5%|████                                                                              | 1/20 [02:56<55:56, 176.65s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\345227218.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SALAD_rocs = SALAD_rocs.append({'fpr': fpr_SALAD, 'tpr': tpr_SALAD}, ignore_index=True)
 10%|████████▏                                                                         | 2/20 [05:57<53:42, 179.03s/it]

...

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\345227218.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SALAD_rocs = SALAD_rocs.append({'fpr': fpr_SALAD, 'tpr': tpr_SALAD}, ignore_index=True)
 90%|███████████████████████████████████████████████████████████████████▌       | 18/20 [11:28:05<3:12:44, 5782.36s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\345227218.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SALAD_rocs = SALAD_rocs.append({'fpr': fpr_SALAD, 'tpr': tpr_SALAD}, ignore_index=True)
 95%|███████████████████████████████████████████████████████████████████████▎   | 19/20 [11:30:58<1:08:17, 4097.65s/it]

675/675 [==============================] - 1s 1ms/step


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\345227218.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SALAD_rocs = SALAD_rocs.append({'fpr': fpr_SALAD, 'tpr': tpr_SALAD}, ignore_index=True)
100%|█████████████████████████████████████████████████████████████████████████████| 20/20 [11:34:15<00:00, 2082.77s/it]


In [27]:
# k-SA-CWoLa
fpr_interp = [None] * len(ksacwola_rocs)

for i, row in ksacwola_rocs.iterrows():
    fpr, tpr = row
    interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

    fpr_pts = interp(tpr_pts)
    fpr_interp[i] = fpr_pts

fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

ksacwola_max = np.nanmax(fpr_interp, axis=0).data
ksacwola_min = np.nanmin(fpr_interp, axis=0).data
ksacwola_med = np.median(fpr_interp, axis=0).data
ksacwola_mean = np.nanmean(fpr_interp, axis=0).data
ksacwola_std = np.nanstd(fpr_interp, axis=0).data
    
sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
ksacwola_smean = np.nanmean(sig_interp, axis=0).data
ksacwola_sstd = np.nanstd(sig_interp, axis=0).data
    
fpr_interp = [None] * len(SALAD_rocs)

for i, row in SALAD_rocs.iterrows():
    fpr, tpr = row
    interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

    fpr_pts = interp(tpr_pts)
    fpr_interp[i] = fpr_pts

fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

SALAD_max = np.nanmax(fpr_interp, axis=0).data
SALAD_min = np.nanmin(fpr_interp, axis=0).data
SALAD_med = np.median(fpr_interp, axis=0).data
SALAD_mean = np.nanmean(fpr_interp, axis=0).data
SALAD_std = np.nanstd(fpr_interp, axis=0).data
 
sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
SALAD_smean = np.nanmean(sig_interp, axis=0).data
SALAD_sstd = np.nanstd(sig_interp, axis=0).data

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1917551772.py:11: RuntimeWarning: divide by zero encountered in true_divide
  fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))
C:\Users\bolit\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1917551772.py:33: RuntimeWarning: divide by zero encountered in true_divide
  fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))


In [28]:
# Autoencoder
dataset_bg_inSR = dataset_bg[sr_crit(dataset_bg)]
dataset_sig_inSR = dataset_sig[sr_crit(dataset_sig)]
ae_rocs = pd.DataFrame(columns=['fpr', 'tpr'])

for i in tqdm.tqdm(range(n_injections)):
    
    X_ae_train = dataset_bg_inSR[0:int(0.5*len(dataset_bg_inSR)),1:]
    X_ae_val = np.concatenate([dataset_bg_inSR[int(0.5*len(dataset_bg_inSR)):, 1:],
                               dataset_sig_inSR[:, 1:]])
    Y_ae_val = np.concatenate([np.zeros(len(dataset_bg_inSR[int(0.5*len(dataset_bg_inSR)):, 1:])),
                               np.ones(len(dataset_sig_inSR[:, 1:]))])
    
    model_ae = Sequential()
    model_ae.add(Dense(64, input_dim=4, activation='relu'))
    #model_ae.add(Dense(64, activation='relu'))
    model_ae.add(Dense(2, activation='relu'))
    model_ae.add(Dense(64, activation='relu'))
    #model_ae.add(Dense(64, activation='relu'))
    model_ae.add(Dense(4, activation='linear'))
    model_ae.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    hist_ae = model_ae.fit(X_ae_train, X_ae_train, epochs=EPOCHS, 
                           batch_size=batch_size, verbose=0)  
    
    ae_save_folder = 'models/autoencoder/'
    if not os.path.exists(ae_save_folder):
        os.makedirs(ae_save_folder)
    model_ae.save('models/autoencoder/model_%i'%i)
    
    scores_ae = model_ae.predict(X_ae_val,batch_size=batch_size,verbose=0)
    mses_ae = np.linalg.norm(scores_ae-X_ae_val,axis=1)
    
    if (False):
        plt.hist(mses_ae[Y_sup_train==0],bins=np.linspace(0,1,20),alpha=0.5,label="Background",density=True)
        plt.hist(mses_ae[Y_sup_train==1],bins=np.linspace(0,1,20),alpha=0.5,label="Signal",density=True)
        plt.xlabel("MSE")
        plt.legend(frameon=False)
        
    fpr_ae, tpr_ae, _ = roc_curve(Y_ae_val, mses_ae)
    ae_rocs = ae_rocs.append({'fpr': fpr_ae, 'tpr': tpr_ae}, ignore_index=True)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: models/autoencoder/model_0\assets


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1320818217.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ae_rocs = ae_rocs.append({'fpr': fpr_ae, 'tpr': tpr_ae}, ignore_index=True)
  5%|████▏                                                                              | 1/20 [00:12<04:06, 12.99s/it]

INFO:tensorflow:Assets written to: models/autoencoder/model_1\assets


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1320818217.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ae_rocs = ae_rocs.append({'fpr': fpr_ae, 'tpr': tpr_ae}, ignore_index=True)
 10%|████████▎                                                                          | 2/20 [00:25<03:50, 12.82s/it]

...

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1320818217.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ae_rocs = ae_rocs.append({'fpr': fpr_ae, 'tpr': tpr_ae}, ignore_index=True)
 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [03:52<00:25, 12.55s/it]

INFO:tensorflow:Assets written to: models/autoencoder/model_18\assets


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1320818217.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ae_rocs = ae_rocs.append({'fpr': fpr_ae, 'tpr': tpr_ae}, ignore_index=True)
 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:04<00:12, 12.42s/it]

INFO:tensorflow:Assets written to: models/autoencoder/model_19\assets


C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\1320818217.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ae_rocs = ae_rocs.append({'fpr': fpr_ae, 'tpr': tpr_ae}, ignore_index=True)
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:16<00:00, 12.85s/it]


In [29]:
# autoencoder
fpr_interp = [None] * len(ae_rocs)
    
for i, row in ae_rocs.iterrows():
    fpr, tpr = row
    interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

    fpr_pts = interp(tpr_pts)
    fpr_interp[i] = fpr_pts

fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

ae_max = np.nanmax(fpr_interp, axis=0).data
ae_min = np.nanmin(fpr_interp, axis=0).data
ae_med = np.median(fpr_interp, axis=0).data
ae_mean = np.nanmean(fpr_interp, axis=0).data
ae_std = np.nanstd(fpr_interp, axis=0).data
    
sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
ae_smean = np.nanmean(sig_interp, axis=0).data
ae_sstd = np.nanstd(sig_interp, axis=0).data

C:\Users\bolit\AppData\Local\Temp\ipykernel_13380\2450094283.py:11: RuntimeWarning: divide by zero encountered in true_divide
  fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))
C:\Users\bolit\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [30]:
#First, let's do fully supervised

Nsig_each = int(len(SR_signal)/2)

SR_sig_draw = np.random.permutation(SR_signal)
X_sup_train = np.concatenate([SR_sig_draw[:Nsig_each,1:],SR_background_data[:,1:]])
Y_sup_train = np.concatenate([np.ones(len(SR_sig_draw[:Nsig_each,1:])),np.zeros(len(SR_background_data))])

X_sup_val = np.concatenate([SR_sig_draw[Nsig_each:,1:],SR_background_data_fortest[:,1:]])
Y_sup_val = np.concatenate([np.ones(len(SR_sig_draw[Nsig_each:,1:])),np.zeros(len(SR_background_data_fortest))])

model_sup = Sequential()
model_sup.add(Dense(64, input_dim=N_inputs, activation='relu')) 
model_sup.add(Dense(64, activation='relu'))
model_sup.add(Dense(64, activation='relu'))
model_sup.add(Dense(1, activation='sigmoid'))
model_sup.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist_sup = model_sup.fit(X_sup_train, Y_sup_train, epochs=EPOCHS, batch_size=batch_size, verbose=1)

scores_sup = model_sup.predict(X_sup_val,batch_size=batch_size,verbose=0)
fpr_sup, tpr_sup, _ = roc_curve(Y_sup_val, scores_sup)

Epoch 1/20
492/492 [==============================] - 2s 2ms/step - loss: 0.3614 - accuracy: 0.8466
Epoch 2/20
492/492 [==============================] - 1s 2ms/step - loss: 0.3148 - accuracy: 0.8743
Epoch 3/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2992 - accuracy: 0.8817
Epoch 4/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2919 - accuracy: 0.8849
Epoch 5/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2838 - accuracy: 0.8891
Epoch 6/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2737 - accuracy: 0.8938
Epoch 7/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2650 - accuracy: 0.8983
Epoch 8/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2560 - accuracy: 0.9025
Epoch 9/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2493 - accuracy: 0.9052
Epoch 10/20
492/492 [==============================] - 1s 2ms/step - loss: 0.2468 - accuracy: 0.9054

In [33]:
def load_data (path, name='dataset', train=None, test=None, signal=None, background=None, sample=None, seed=21, replace=True):
    """
    General script to load data, common to all run scripts.

    Arguments:
        path: The path to the HDF5 file, from which data should be loaded.
        name: Name of the dataset, as stored in the HDF5 file.
        ...

    Returns:
        Tuple of pandas.DataFrame containing the loaded; list of loaded features
        to be used for training; and list of features to be used for mass-
        decorrelation.

    Raises:
        IOError: If no HDF5 file exists at the specified `path`.
        KeyError: If the HDF5 does not contained a dataset named `name`.
        KeyError: If any of the necessary features are not present in the loaded
            dataset.
    """

    # Check(s)
    #assert False not in [train, test, signal, background]
    #if sample: assert 0 < sample and sample < 1.

    # Read data from HDF5 file
    data = pd.read_hdf(path)

    # Subsample signal by x10 for testing: 1E+07 -> 1E+06
    np.random.seed(7)
    try:
        msk_test  = data['train'] == 0
        msk_train = ~msk_test
        msk_bkg = data['signal'] == 0
        msk_sig = ~msk_bkg
        #idx_sig = np.where(msk_sig)[0]
        #idx_sig = np.random.choice(idx_sig, int(msk_sig.sum() * 0.1), replace=False)
        #msk_sig = np.zeros_like(msk_bkg).astype(bool)
        #msk_sig[idx_sig] = True
        #data = data[msk_train | (msk_test & (msk_sig | msk_bkg))]
    except:
        log.warning("Some of the keys ['train', 'signal'] were not present in file {}".format(path))
        pass



    # Define feature collections to use
    features_input = ['Tau21pJ1', 'Tau21J2', 'Tau32J1', 'Tau32J2']
    features_decorrelation = ['m']

    # Split data
    if train:
        log.info("load_data: Selecting only training data.")
        data = data[data['train']  == 1]
        pass

    if test:
        # log.info("load_data: Selecting only testing data.")
        data = data[data['train']  == 0]
        pass

    if signal:
        log.info("load_data: Selecting only signal data.")
        data = data[data['signal'] == 1]
        pass

    if background:
        log.info("load_data: Selecting only background data.")
        data = data[data['signal'] == 0]
        pass

    if sample:
        log.info("load_data: Selecting a random fraction {:.2f} of data (replace = {}, seed = {}).".format(sample, replace, seed))
        data = data.sample(frac=sample, random_state=seed, replace=False)
        pass

    # Return
    return data, features_input, features_decorrelation


In [32]:
def load (basedir, name, model=None):

    # Import(s)
    from keras.models import load_model

    # Load full pre-trained model or model weights
    if model is None:
        model = load_model(basedir + '{}.h5'.format(name))
    else:
        model.load_weights(basedir + '{}_weights.h5'.format(name))
        pass

    # Load associated training histories
    try:
        history_file = basedir + 'history__{}.json'.format(name)
        with open(history_file, 'r') as f:
            history = json.load(f)
            pass
    except:
        print ("[WARN] Could not find history file {}.") #
        history = None
        pass

    return model, history

In [31]:

 # Define variable(s)
name    = 'classifier'
basedir = '/Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoLa_ANN_Analysis/hybrid_method/output'

ann_rocs = pd.DataFrame(columns=['fpr', 'tpr'])



data, features, features_decorrelation = load_data('/Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoLa_ANN_Analysis/DataF.h5', test=True)

# Prepare arrays
X = data[features].values
Y = data['signal'].values


classifier, history = load(basedir, name)

# Predict
scores_ann = classifier.predict(X, batch_size = 1024) #X reemplazar por datos CWola


fpr_ann, tpr_ann, _ = roc_curve(Y, scores_ann)


ann_rocs = ann_rocs.append({'fpr': fpr_ann, 'tpr': tpr_ann}, ignore_index=True)



FileNotFoundError: File /Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoLa_ANN_Analysis/DataF.h5 does not exist

In [ ]:
# ANN
fpr_interp = [None] * len(ann_rocs)
    
for i, row in ann_rocs.iterrows():
    fpr, tpr = row
    interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

    fpr_pts = interp(tpr_pts)
    fpr_interp[i] = fpr_pts

fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

ann_max = np.nanmax(fpr_interp, axis=0).data
ann_min = np.nanmin(fpr_interp, axis=0).data
ann_med = np.median(fpr_interp, axis=0).data
ann_mean = np.nanmean(fpr_interp, axis=0).data
ann_std = np.nanstd(fpr_interp, axis=0).data
   
sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
ann_smean = np.nanmean(sig_interp, axis=0).data
ann_sstd = np.nanstd(sig_interp, axis=0).data

In [ ]:
bands = {'cwola': (cwola_max, cwola_med, cwola_min, cwola_mean, cwola_std, cwola_smean, cwola_sstd, {'label':'CWoLa', 'color':'red', 'ls':':'}),
         #'dvsim': (dvsim_max, dvsim_med, dvsim_min, dvsim_mean, dvsim_std, dvsim_smean, dvsim_sstd, {'label':'Data vs. Sim', 'color':'blue', 'ls':':'}),
        'ocwola': (ocwola_max, ocwola_med, ocwola_min, ocwola_mean, ocwola_std, ocwola_smean, ocwola_sstd, {'label':'Optimal CWoLa', 'color':'red'}),
          'autoencoder': (ae_max, ae_med, ae_min, ae_mean, ae_std, ae_smean, ae_sstd, {'label':"Autoencoder".format(1), 'color':'blue'}),
          'SALAD': (SALAD_max, SALAD_med, SALAD_min, SALAD_mean, SALAD_std, SALAD_smean, SALAD_sstd, {'label':'SALAD', 'color':'tab:purple'}),
          'ANN': (ann_max, ann_med, ann_min, ann_mean, ann_std, ann_smean, ann_sstd, {'label':'ANN', 'color':'tab:orange'}),
          'sacwola': (ksacwola_max, ksacwola_med, ksacwola_min, ksacwola_mean, ksacwola_std, ksacwola_smean, ksacwola_sstd, {'label':"SA-CWoLa, $\lambda = {}$".format(myklambda), 'color':'black'}),
          }

In [ ]:
# Plot without banding
mylambda=.5
myklambda = 1 #np.round(len(SR_background_data)/len(SR_background_sim), 3)

fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1])
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.yscale("log")
plt.ylim([1,5*1e4])
ERR=False
plt.plot(tpr_sup,1./fpr_sup,label="Supervised",color='green')

for band in bands:
    if band in ['old-sacwola', 'hl-sacwola']:
        continue
    mu,std = bands[band][3:5]

    plt.plot(tpr_pts, mu, **bands[band][-1])
    if ERR:
        plt.fill_between(tpr_pts, mu - std, mu + std, alpha=.2, color=bands[band][-1]['color'])
#plt.plot(tpr_sup,1./tpr_sup,color="black",ls=":",label="Random")

plt.xlabel(r'Signal Efficiency (True Positive Rate)',fontsize=20)
plt.ylabel(r"Rejection (1/False Positive Rate)",fontsize=20)
plt.legend(frameon=False,fontsize=15,loc="upper right")

# plt.title("Correlated LHCO Example\n(Median of {} models)".format(n_injections),fontsize=20,)
#plt.savefig('C:/Users/knukl/Desktop/FINAL/plots/combined_corr_deltamj_extracorr.pdf', pdf=True, bbox_inches='tight')
#plt.savefig('C:/Users/knukl/Desktop/FINAL/plots/combined_corr_deltamj_extracorr.png', pdf=False, bbox_inches='tight')
# plt.show()

In [ ]:
# Plot without banding
mylambda=.5
myklambda = 1 #np.round(len(SR_background_data)/len(SR_background_sim), 3)

ERR = False

fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1])
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.ylim([0, 6])



for band in bands:
    if band in ['old-sacwola', 'hl-sacwola']:
        continue
    mu,std = bands[band][5:7]
    plt.plot(tpr_pts, tpr_pts*mu, **bands[band][-1])
plt.plot(tpr_sup,tpr_sup/np.sqrt(tpr_sup),color="black",ls=":",label="Random")

plt.xlabel(r'Signal Efficiency (True Positive Rate)',fontsize=20)
plt.ylabel(r"Significance improvement",fontsize=20)
plt.legend(frameon=False,fontsize=10,loc="upper right")
#plt.title(r'$LHC$ $Olympics$ $Example$', loc='right', fontsize=20)

#plt.savefig('C:/Users/knukl/Desktop/FINAL/plots/combined_relsigs_corr_deltamj_extracorr.pdf', pdf=True, bbox_inches='tight')
#plt.savefig('C:/Users/knukl/Desktop/FINAL/plots/combined_relsigs_corr_deltamj_extracorr.png', pdf=False, bbox_inches='tight')